# Elegant and efficient NumPy

This tutorial is for those who have clobbered together an analysis script or two, using basic Python and NumPy, and want to learn how to write more elegant and efficient analysis code. We will dive into how NumPy works behind the scenes and use this knowledge to our advantage. This tutorial also serves as an introduction to reading the definitive work on this topic: [Guide to NumPy](http://web.mit.edu/dvp/Public/numpybook.pdf) by Travis E. Oliphant, the initial creator of NumPy.

## Why can NumPy be so fast?
Python, being an interpreted programming language, is quite slow. Manipulating large amounts of numbers using Python's build-in lists would be impractically slow for any serious data analysis. Yet, the `numpy` package can be really fast. What gives?

How fast can NumPy be? Let's try summing together 10000000 random numbers in a fast language, C, and using NumPy. First, the C version that uses a naive approach of doing this:

In [252]:
%%writefile random.c
#include <stdlib.h>
#include <stdio.h>
#define N_ELEMENTS 10000000

int main(int argc, char** argv) {
    double* a = (double*) malloc(sizeof(double) * N_ELEMENTS);
    
    int i;
    for(i=0; i<N_ELEMENTS; ++i) {
        a[i] = (double) rand() / RAND_MAX;
    }
    
    double sum = 0;
    for(i=0; i<N_ELEMENTS; ++i) {
        sum += a[i];
    }

    printf("%f", sum);
    return 0;
}

Overwriting random.c


The above snippet of C code creates an array of 10000000 64-bit floating point numbers, chosen randomly between 0 and 1, adds them all together, and displays the result.

The cell below will compile the above code using the default C compiler, using maximum optimization settings.
On Linux and OSX systems, a compiler should be readily available.
On Windows, you may need to install one and modify the code below to use it.

In [253]:
!gcc random.c -o random -Ofast

With the program compiled and ready to run, let's see how long it takes. Here, I'm using the `%%timeit` magic cell function to instruct the Jupyter Notebook to run the program 10 times and report the average running time:

In [254]:
%%timeit -r 10 -n 1
!random

4999634.344218
4999634.344218
4999634.344218
4999634.344218
4999634.344218
4999634.344218
4999634.344218
4999634.344218
4999634.344218
4999634.344218
399 ms ± 50.6 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


### Beating C-code with NumPy
<img src="images/race.jpg" style="float: right; width: 300px; margin: 10px; margin-top: -10px"/>
On my laptop, the C version took ±387 ms to run. How long did it take on the machine you are currently using?

As a test of your current knowledge of NumPy, can you write a little snippet of Python + NumPy code that performs the same operations, but faster than the naive C program above? To recap, your program should:

 1. Create an array of 10000000 random values, each value between 0 and 1.
 2. Compute and print the sum of the array. It's ok if you get a different result than the C code. They are random numbers after all.
 
We can use the `%%timeit` magic again to determine the running time of your code.

In [255]:
%%timeit -r 10 -n 1

# Write your Python + NumPy code here and execute the cell to see how long it takes
import numpy as np
np.random.seed(0)
a = np.random.rand(10000000)
print(np.sum(a))

4999995.596966093
4999995.596966093
4999995.596966093
4999995.596966093
4999995.596966093
4999995.596966093
4999995.596966093
4999995.596966093
4999995.596966093
4999995.596966093
231 ms ± 15 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


I hope you found this task easy to accomplish.

How did we beat the C version so easily?
The NumPy module is written in C, so it must be doing something more clever than the naive C code above.
The truth is that behind the scenes, NumPy is using non-naive, highly optimized, C and FORTRAN code.

## The man behind the curtain: Basic Linear Algebra Subprograms
<img src="images/wizard-of-oz.jpg" style="float: right; width: 300px; margin: 10px; margin-top: -10px"/>

[Basic Linear Algebra Subprograms (BLAS)](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms) for Fortran was published in 1979 [1] and is a collection algorithms for common mathematical operations that are performed on arrays of numbers.
Algorithms such as element-wise sum, matrix multiplication, computing the vector length, etc.

The API of that software library was later standardized, and today there are many modern implementations available. These libraries represent over 40 years of optimizing efforts and make use of [specialized CPU instructions for manipulating arrays](https://www.youtube.com/watch?v=Pc8DfEyAxzg&list=PLzLzYGEbdY5lrUYSssHfk5ahwZERojgid).
In other words, they are *fast*.

Hence, we should strive to push as much of the work as possible into NumPy's heavily optimized innards.
Less code for us to write and better performance to boot.
However, this does mean we will have to get intimately familiar with its [API](https://docs.scipy.org/doc/numpy/reference/).
In the following tutorials, we will tour some of NumPy's nooks and crannies that are not necessarily covered in basic tutorials, but are nontheless worthwhile to know about if you are serious about learning data analysis with Python.

[1] C. L. Lawson, R. J. Hanson, D. R. Kincaid, and F. T. Krogh. 1979. Basic Linear Algebra Subprograms for Fortran Usage. ACM Trans. Math. Softw. 5, 3 (September 1979), 308-323. DOI: https://doi.org/10.1145/355841.355847 